In [1]:
# Cell 1: Path & folder setup — run this first
from pathlib import Path
import pandas as pd
import os

# Project root (assume notebook is in notebooks/)
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name.lower() == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
TEMP_DIR = PROJECT_ROOT / "temp_storage"
DOCS_DIR = PROJECT_ROOT / "docs"
IMAGES_DIR = PROJECT_ROOT / "images"

# Use exact merged CSV path you found
MERGED_CSV = Path(r"C:\Users\Biswajit\citi-bike-nyc-2022-dashboard\data\processed\citibike_with_weather_merged.csv")

print("Project root:", PROJECT_ROOT)
print("Processed dir:", PROCESSED_DIR)
print("MERGED_CSV exists:", MERGED_CSV.exists(), MERGED_CSV)

Project root: C:\Users\Biswajit\citi-bike-nyc-2022-dashboard
Processed dir: C:\Users\Biswajit\citi-bike-nyc-2022-dashboard\data\processed
MERGED_CSV exists: True C:\Users\Biswajit\citi-bike-nyc-2022-dashboard\data\processed\citibike_with_weather_merged.csv


In [21]:
fn = "../temp/citibike_merged_2022.csv"   # if stored in temp/
# OR
fn = "../data/citibike_merged_2022.csv"   # if stored in data/
# OR
fn = "citibike_merged_2022.csv"           # if stored next to the notebook
print(fn)

citibike_merged_2022.csv


In [22]:
# Cell 2: load merged dataset
import pandas as pd

# MERGED_CSV is set in Cell 1
print("Loading:", MERGED_CSV)
df = pd.read_csv(MERGED_CSV, low_memory=False,
                 parse_dates=['date'] if 'date' in pd.read_csv(MERGED_CSV, nrows=0).columns else None)

print("Loaded df shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head()

Loading: C:\Users\Biswajit\citi-bike-nyc-2022-dashboard\data\processed\citibike_with_weather_merged.csv
Loaded df shape: (895485, 20)
Columns: ['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual', '_start_dt', 'date', 'value', 'datatype', 'station', 'attributes', '_merge']


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,_start_dt,date,value,datatype,station,attributes,_merge
0,CA5837152804D4B5,electric_bike,2022-01-26 18:50:39,2022-01-26 18:51:53,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-26 18:50:39,2022-01-26,1171.016667,ADPT,GHCND:USW00014732,",,W,",both
1,BA06A5E45B6601D2,classic_bike,2022-01-28 13:14:07,2022-01-28 13:20:23,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member,2022-01-28 13:14:07,2022-01-28,1030.570000,ADPT,GHCND:USW00014732,",,W,",both
2,7B6827D7B9508D93,classic_bike,2022-01-10 19:55:13,2022-01-10 20:00:37,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member,2022-01-10 19:55:13,2022-01-10,1168.655556,ADPT,GHCND:USW00014732,",,W,",both
3,6E5864EA6FCEC90D,electric_bike,2022-01-26 07:54:57,2022-01-26 07:55:22,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-26 07:54:57,2022-01-26,1171.016667,ADPT,GHCND:USW00014732,",,W,",both
4,E24954255BBDE32D,electric_bike,2022-01-13 18:44:46,2022-01-13 18:45:43,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-13 18:44:46,2022-01-13,1145.411111,ADPT,GHCND:USW00014732,",,W,",both


In [7]:
# Cell 3: create value and aggregate trips between stations
# Normalize column names if slightly different
cols = df.columns.str.lower().tolist()

# common names we expect
start_name_col = next((c for c in df.columns if c.lower() in ('start_station_name','from_station_name','start_station')), None)
end_name_col   = next((c for c in df.columns if c.lower() in ('end_station_name','to_station_name','end_station')), None)

print("Using start station col:", start_name_col)
print("Using end station col:  ", end_name_col)

# add value column if missing
if 'value' not in df.columns:
    df['value'] = 1
else:
    df['value'] = df['value'].fillna(1)

# groupby start->end
df_group = df.groupby([start_name_col, end_name_col])['value'].count().reset_index()
df_group.columns = ['start_station','end_station','trips']   # rename for clarity
df_group = df_group.sort_values('trips', ascending=False).reset_index(drop=True)
print("Aggregated routes:", df_group.shape)
df_group.head(10)

Using start station col: start_station_name
Using end station col:   end_station_name
Aggregated routes: (6953, 3)


,start_station,end_station,trips
0,Hoboken Terminal - Hudson St & Hudson Pl,Hoboken Ave at Monmouth St,5565
1,South Waterfront Walkway - Sinatra Dr & 1 St,South Waterfront Walkway - Sinatra Dr & 1 St,5439
2,Marin Light Rail,Grove St PATH,4113
3,Hoboken Ave at Monmouth St,Hoboken Terminal - Hudson St & Hudson Pl,4083
4,Grove St PATH,Marin Light Rail,3973
5,12 St & Sinatra Dr N,South Waterfront Walkway - Sinatra Dr & 1 St,3964
6,Liberty Light Rail,Liberty Light Rail,3696
7,South Waterfront Walkway - Sinatra Dr & 1 St,12 St & Sinatra Dr N,3495
8,Hamilton Park,Grove St PATH,3203
9,Newport Pkwy,Newport Pkwy,3131


In [23]:
# Cell 4: attach coordinates for start and end (if present)
# try to find lat/lng columns in original df
lat_cols = [c for c in df.columns if 'lat' in c.lower()]
lng_cols = [c for c in df.columns if 'lng' in c.lower() or 'lon' in c.lower()]

print("Latitude columns:", lat_cols)
print("Longitude columns:", lng_cols)

# If original has start_lat/start_lng etc, create station lookup and merge
# We'll build station->lat/lng mapping using start_station columns
station_lat_col = next((c for c in df.columns if 'start_lat' in c.lower()), None)
station_lng_col = next((c for c in df.columns if 'start_lng' in c.lower() or 'start_lon' in c.lower()), None)

if station_lat_col and station_lng_col:
    # station mapping from the original
    start_map = df[[start_name_col, station_lat_col, station_lng_col]].drop_duplicates().rename(
        columns={start_name_col:'start_station', station_lat_col:'start_lat', station_lng_col:'start_lng'}
    )
    end_map = df[[end_name_col, station_lat_col.replace('start','end') if station_lat_col.replace('start','end') in df.columns else station_lat_col, 
                  station_lng_col.replace('start','end') if station_lng_col.replace('start','end') in df.columns else station_lng_col]].drop_duplicates()
    # handle end_map naming generically if columns exist
    if end_name_col in df.columns and ('end_lat' in df.columns or 'start_lat' in df.columns):
        # attempt safer approach: build a station-level table using all station names and coords
        station_tbl = pd.concat([
            df[[start_name_col, station_lat_col, station_lng_col]].rename(columns={start_name_col:'station', station_lat_col:'lat', station_lng_col:'lng'}),
            df[[end_name_col, station_lat_col, station_lng_col]].rename(columns={end_name_col:'station', station_lat_col:'lat', station_lng_col:'lng'})
        ], ignore_index=True).drop_duplicates(subset=['station'])
        station_tbl = station_tbl.dropna(subset=['lat','lng'])
        # create mappings
        df_group = df_group.merge(station_tbl.rename(columns={'station':'start_station','lat':'start_lat','lng':'start_lng'}), on='start_station', how='left')
        df_group = df_group.merge(station_tbl.rename(columns={'station':'end_station','lat':'end_lat','lng':'end_lng'}), on='end_station', how='left')
    else:
        # fallback: try to merge using start and end coordinate columns with best-effort names
        df_group = df_group.merge(start_map, on='start_station', how='left')
        # attempt to build end_map from end columns if present
        end_lat_col = next((c for c in df.columns if 'end_lat' in c.lower()), None)
        end_lng_col = next((c for c in df.columns if 'end_lng' in c.lower() or 'end_lon' in c.lower()), None)
        if end_lat_col and end_lng_col:
            end_map = df[[end_name_col, end_lat_col, end_lng_col]].drop_duplicates().rename(
                columns={end_name_col:'end_station', end_lat_col:'end_lat', end_lng_col:'end_lng'}
            )
            df_group = df_group.merge(end_map, on='end_station', how='left')
else:
    # No start_lat available; try to read stations file in data/raw or data/lookup
    cand_stations = list((pathlib.Path.cwd() / 'data' / 'raw').glob('*station*')) + list((pathlib.Path.cwd() / 'data').glob('*stations*'))
    cand_stations = [p for p in cand_stations if p.is_file()]
    if cand_stations:
        print("Found station lookup:", cand_stations[0])
        stations = pd.read_csv(cand_stations[0])
        # try to guess columns for join
        station_name_col = next((c for c in stations.columns if 'name' in c.lower()), None)
        lat_col = next((c for c in stations.columns if 'lat' in c.lower()), None)
        lon_col = next((c for c in stations.columns if 'lon' in c.lower() or 'lng' in c.lower()), None)
        print("station lookup cols:", station_name_col, lat_col, lon_col)
        stations = stations[[station_name_col, lat_col, lon_col]].rename(columns={station_name_col:'start_station', lat_col:'start_lat', lon_col:'start_lng'})
        df_group = df_group.merge(stations, on='start_station', how='left')
        # duplicate for end (rename then merge)
        stations_end = stations.rename(columns={'start_station':'end_station','start_lat':'end_lat','start_lng':'end_lng'})
        df_group = df_group.merge(stations_end, on='end_station', how='left')

# Final check for coordinate availability
print("Missing start coords:", df_group['start_lat'].isna().sum() if 'start_lat' in df_group.columns else 'no start_lat')
print("Missing end coords:", df_group['end_lat'].isna().sum() if 'end_lat' in df_group.columns else 'no end_lat')
df_group.head()

Latitude columns: ['start_lat', 'end_lat']
Longitude columns: ['start_lng', 'end_lng']
Missing start coords: no start_lat
Missing end coords: no end_lat


,start_station,end_station,trips,start_lat_x,start_lng_x,end_lat_x,end_lng_x,start_lat_y,start_lng_y,end_lat_y,end_lng_y
0,Hoboken Terminal - Hudson St & Hudson Pl,Hoboken Ave at Monmouth St,5565,40.735938,-74.030305,40.735208,-74.046964,40.735938,-74.030305,40.735208,-74.046964
1,South Waterfront Walkway - Sinatra Dr & 1 St,South Waterfront Walkway - Sinatra Dr & 1 St,5439,40.736982,-74.027781,40.736982,-74.027781,40.736982,-74.027781,40.736982,-74.027781
2,Marin Light Rail,Grove St PATH,4113,40.714584,-74.042817,40.719586,-74.043117,40.714584,-74.042817,40.719586,-74.043117
3,Hoboken Ave at Monmouth St,Hoboken Terminal - Hudson St & Hudson Pl,4083,40.735208,-74.046964,40.735938,-74.030305,40.735208,-74.046964,40.735938,-74.030305
4,Grove St PATH,Marin Light Rail,3973,40.719586,-74.043117,40.714584,-74.042817,40.719586,-74.043117,40.714584,-74.042817


In [9]:
# Cell 5: create df_final for mapping; drop rows missing coords
needed_cols = ['start_station','end_station','trips','start_lat','start_lng','end_lat','end_lng']
for c in needed_cols:
    if c not in df_group.columns:
        df_group[c] = np.nan

df_final = df_group.dropna(subset=['start_lat','start_lng','end_lat','end_lng']).copy()
print("Rows with full coords:", df_final.shape)
df_final = df_final.astype({'start_lat':float,'start_lng':float,'end_lat':float,'end_lng':float,'trips':int})
df_final.head()

Rows with full coords: (6953, 7)


,start_station,end_station,trips,start_lat,start_lng,end_lat,end_lng
0,Hoboken Terminal - Hudson St & Hudson Pl,Hoboken Ave at Monmouth St,5565,40.735938,-74.030305,40.735208,-74.046964
1,South Waterfront Walkway - Sinatra Dr & 1 St,South Waterfront Walkway - Sinatra Dr & 1 St,5439,40.736982,-74.027781,40.736982,-74.027781
2,Marin Light Rail,Grove St PATH,4113,40.714584,-74.042817,40.719586,-74.043117
3,Hoboken Ave at Monmouth St,Hoboken Terminal - Hudson St & Hudson Pl,4083,40.735208,-74.046964,40.735938,-74.030305
4,Grove St PATH,Marin Light Rail,3973,40.719586,-74.043117,40.714584,-74.042817


In [24]:
# Create df_final for kepler arcs/lines
import numpy as np

# Inspect column names to ensure correct column mapping
print("Columns in df:", df.columns.tolist())

# Common column names in your dataset (adjust if different)
# 'start_station_id','end_station_id','start_lat','start_lng','end_lat','end_lng'
# If your dataset uses other column names (e.g., 'start_latitude') update below accordingly.

# First make sure coordinates exist
coords_cols = {'start_lat', 'start_lng', 'end_lat', 'end_lng'}
if not coords_cols.issubset(set(df.columns)):
    print("WARNING: expected coord columns missing. Found:", [c for c in df.columns if 'lat' in c or 'lng' in c])
    # If your columns are named differently, map them here. Example:
    # df = df.rename(columns={'start_latitude':'start_lat', 'start_longitude':'start_lng', ...})

# Create a 'value' column to count trips if not present
if 'value' not in df.columns:
    df['value'] = 1

# Group by start/end station name or id to aggregate trips
group_cols = []
if 'start_station_name' in df.columns and 'end_station_name' in df.columns:
    group_cols = ['start_station_name', 'end_station_name']
elif 'start_station_id' in df.columns and 'end_station_id' in df.columns:
    group_cols = ['start_station_id', 'end_station_id']
else:
    # fallback to lat/lng pairs
    group_cols = ['start_lat', 'start_lng', 'end_lat', 'end_lng']

agg = df.groupby(group_cols).agg({
    'value': 'sum',
    'start_lat': 'first' if 'start_lat' in df.columns else 'mean',
    'start_lng': 'first' if 'start_lng' in df.columns else 'mean',
    'end_lat': 'first' if 'end_lat' in df.columns else 'mean',
    'end_lng': 'first' if 'end_lng' in df.columns else 'mean'
}).reset_index().rename(columns={'value':'trips'})

# If the group_cols were station names, ensure lat/lng columns exist (they should via aggregation)
df_final = agg.copy()
print("df_final shape:", df_final.shape)
df_final.head()

Columns in df: ['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual', '_start_dt', 'date', 'value', 'datatype', 'station', 'attributes', '_merge']
df_final shape: (6953, 7)


,start_station_name,end_station_name,trips,start_lat,start_lng,end_lat,end_lng
0,11 St & Washington St,11 St & Washington St,78674.325464,40.749985,-74.02715,40.749985,-74.027150
1,11 St & Washington St,12 Ave & W 40 St,0.000000,40.749985,-74.02715,40.760875,-74.002777
2,11 St & Washington St,12 St & Sinatra Dr N,8003.942222,40.749985,-74.02715,40.750604,-74.024020
3,11 St & Washington St,14 St Ferry - 14 St & Shipyard Ln,13610.980234,40.749985,-74.02715,40.752961,-74.024353
4,11 St & Washington St,4 St & Grand St,26997.812160,40.749985,-74.02715,40.742258,-74.035111


In [25]:
from keplergl import KeplerGl

# Create the Kepler map using df_final
m = KeplerGl(height=700, data={"bike_trips": df_final})
# In notebook environments kepler displays inline; simply returning m will render in JupyterLab/Jupyter notebook (VS Code supports this)
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'bike_trips':          start_station_name                   end_station_name         trips  \
0…

In [26]:
# Cell 6: initialize Kepler map and add our df_final
m = KeplerGl(height=700, data={"bike_trips": df_final})
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'bike_trips':          start_station_name                   end_station_name         trips  \
0…

In [28]:
# Save Kepler config + HTML (run after Kepler map 'm' exists)
from pathlib import Path
import json

# ensure docs folder exists
Path("docs").mkdir(exist_ok=True)

# capture current GUI settings
config = m.config

# write config JSON to docs/
with open("docs/kepler_config.json", "w", encoding="utf8") as f:
    json.dump(config, f)

# save the interactive HTML with config embedded (overwrite existing if present)
out_html = "Divvy_Bike_Trips_Aggregated.html"  # keep this as the single canonical map file
m.save_to_html(file_name=out_html, read_only=False, config=config)

print("Saved docs/kepler_config.json and", out_html)

Map saved to Divvy_Bike_Trips_Aggregated.html!
Saved docs/kepler_config.json and Divvy_Bike_Trips_Aggregated.html


In [27]:
import json
from pathlib import Path

Path("docs").mkdir(exist_ok=True)
config = m.config

# Save config JSON and HTML with config embedded
with open("docs/kepler_config.json", "w", encoding="utf8") as f:
    json.dump(config, f)

m.save_to_html(file_name="docs/citibike_map.html", read_only=False, config=config)
print("Saved docs/kepler_config.json and docs/citibike_map.html")

Map saved to docs/citibike_map.html!
Saved docs/kepler_config.json and docs/citibike_map.html


In [ ]:
# display saved map inside notebook using an IFrame
from IPython.display import IFrame, display
display(IFrame(src="../docs/Divvy_Bike_Trips_Aggregated.html", width=1000, height=700))

In [20]:
# Cell 7: Save the configuration and export to HTML
config = m.config  # capture current GUI settings (after you configure)
# write config to file
with open("kepler_config.json", "w") as f:
    json.dump(config, f)

# Save interactive HTML (this is a copy you can send to mentor)
m.save_to_html(file_name='Divvy_Bike_Trips_Aggregated.html', read_only=False, config=config)
print("Saved kepler_config.json and Divvy_Bike_Trips_Aggregated.html")

Map saved to Divvy_Bike_Trips_Aggregated.html!
Saved kepler_config.json and Divvy_Bike_Trips_Aggregated.html


## Kepler map customization notes
- **Layers enabled:** Points for start stations, Points for end stations, Arc (start -> end).
- **Encodings:** Arc color set to `trips` (quantitative) using a gradient palette so busiest routes are brightest; point color set to neutral single color.
- **Size:** Arc thickness mapped to `trips` to emphasize major corridors.
- **Filter used:** numeric filter on `trips` to show routes with trips >= [YOUR_THRESHOLD].
- **Why:** This highlights frequently used routes and reduces clutter from one-off trips.